In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import precision_score
import pickle

In [2]:
data = pd.read_csv("data.csv")
data

,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share,...,590,591,592,593,594,595,596,597,598,599
0,can we ever store energy produced in lightning,is it possible to store the energy of lightning,1,46,47,8,9,3,17,0.18,...,-0.064226,-0.007377,-0.098092,0.037903,0.023961,0.049796,0.017962,-0.011475,0.050546,0.009905
1,where can i get flexible horse fence solution ...,where can i found very flexible horse fence sy...,1,55,68,10,12,7,22,0.32,...,-0.117809,0.109020,0.003110,0.114025,-0.030353,0.038874,0.063665,-0.026331,-0.027166,-0.021326
2,how should i prepare for sbi po 2017,how do i prepare for sbi po 2017,1,36,32,8,8,7,16,0.44,...,-0.100500,-0.045642,-0.150977,0.032715,-0.110229,-0.048122,0.000171,-0.009717,0.000549,-0.008447
3,what are the reasons why many black people hav...,why do some black people have yellow sclera,1,59,43,11,8,5,19,0.26,...,-0.046242,0.015282,-0.103561,0.007416,-0.008728,0.032753,0.053482,-0.005829,0.093458,-0.039566
4,what were the major effects of the cambodia ea...,what were the major effects of the cambodia ea...,1,124,120,22,22,16,36,0.44,...,-0.139042,-0.025841,-0.039823,0.031169,-0.034440,0.028170,-0.042912,0.004785,0.054871,-0.027787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199989,what laptop is best for deep learning experiments,what is the best laptop i can get to learn dee...,0,49,66,8,14,6,22,0.27,...,-0.094882,0.008527,-0.108083,-0.046972,-0.060542,-0.014592,0.027044,-0.035248,0.027751,0.014219
199990,can i ride horses in worldcraft android,how can you get a town hall 9 account for free,0,40,46,8,11,1,19,0.05,...,-0.060259,0.027296,-0.115234,-0.041721,-0.035177,-0.031373,0.031362,-0.071615,-0.034004,-0.049452
199991,which anime are a must watch,what are the best anime series to watch i hav...,0,28,78,6,17,3,21,0.14,...,-0.065103,-0.028367,0.000352,0.022705,-0.008188,-0.065932,0.004169,0.036800,0.040282,-0.060434
199992,what is the highest temperature a human being ...,what is the highest temperature someone has ha...,0,57,57,10,10,5,20,0.25,...,-0.122938,0.008640,-0.133877,0.140828,-0.041233,-0.001580,-0.026204,-0.060730,0.029093,-0.041748


In [4]:
X = data.iloc[:, 3:]
y = data["is_duplicate"]

In [15]:
scaling_cols = ['q1_len', 'q2_len', 'q1_num_words', 'q2_num_words', 'word_common', 'word_total', 'word_share', 'abs_len_diff', 'mean_len', 'longest_substr_ratio', 'fuzz_ratio', 'fuzz_partial_ratio', 'token_sort_ratio', 'token_set_ratio']

In [16]:
scaler = ColumnTransformer([("standard", StandardScaler(), scaling_cols)], remainder="passthrough")
scaler.fit(X)

ColumnTransformer(remainder='passthrough',
                  transformers=[('standard', StandardScaler(),
                                 ['q1_len', 'q2_len', 'q1_num_words',
                                  'q2_num_words', 'word_common', 'word_total',
                                  'word_share', 'abs_len_diff', 'mean_len',
                                  'longest_substr_ratio', 'fuzz_ratio',
                                  'fuzz_partial_ratio', 'token_sort_ratio',
                                  'token_set_ratio'])])

In [17]:
X_scaled = scaler.transform(X)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=10, test_size=0.15, shuffle=True)

In [23]:
y_train = y_train.to_numpy().reshape(-1,)
y_test = y_test.to_numpy().reshape(-1,)

In [38]:
model = RandomForestClassifier(n_estimators=150, n_jobs=-1, max_depth=38)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=38, n_estimators=150, n_jobs=-1)

In [46]:
y_pred = model.predict(X_test)
precision_score(y_test, y_pred)

0.7528343392182301

In [49]:
with open('rfc.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

In [50]:
with open("scaler.pkl", "wb") as s:
    pickle.dump(scaler, s)